In [1]:
import glob
from pathlib import Path
from experiments import *
from metrics import *
from config import *
from fastai.vision import *
from dataset import pad_tensor, addIgnore

%load_ext autoreload
%autoreload 2

In [32]:
random_seed(42)
CSV_PATH = EXPERIMENTS_PATH + '/BDUNET/predictions.csv'

folds = list(KFold(n_splits = 5, shuffle = True, random_state=42).split(trainFolders, trainFolders))

for idx, (_, valid_indexes) in enumerate(folds):
    if idx not in [1, 2]:
        m = MetricsCallback(None)
        m.on_train_begin() 
        images = []
        for f in glob.glob(MASKS_PATH + "/**/*.cache"):
            index = trainFolders.index(Path(f).parent.name)
            if index in valid_indexes:
                pred = open_mask(BDUNET_PATH + '/' + Path(f).parent.name + '/' + Path(f).stem + '.png')
                pred.px = pred.px != 0
                seg = open_mask(f)
                m.on_batch_end(False, pad_tensor(pred.px, 8), seg.px[:, 0:pred.px.shape[1], 0:pred.px.shape[2]])
        m.calculateMetrics()
        m.save(CSV_PATH, idx > 0)